In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [2]:
# Training settings
batch_size = 64
test_batch_size = 1000
epochs = 14
lr = 1.0
gamma = 0.7
seed = 1
log_interval = 10
use_cuda = torch.cuda.is_available()


In [3]:
torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)


In [4]:

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.337211
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.282871
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.878778
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.743769
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.757818
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.503064
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.195471
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.313188
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.288417
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.262936
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.367791
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.190467
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.229070
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.293686
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.257620
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.168432
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.336814
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.287663
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.359302
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.157020
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.101959
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.093077
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.021877
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.151452
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.007995
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.075905
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.019563
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.083094
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.009057
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.056150
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.032813
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.115503
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.066862
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.010297
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.051766
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.242714
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.061568
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.019002
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.023277
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.082029


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.041673
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.019900
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.029296
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.001830
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.029762
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.006000
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.027527
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.034065
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.384374
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.138217
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.050443
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.030991
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.038775
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.019672
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.117269
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.081160
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.227801
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.108337
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.017759
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.045821


Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.003917
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.036174
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.030585
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.024797
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.033441
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.013622
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.009754
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.031371
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.044816
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.018213
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.005740
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.010478
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.071229
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.003978
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.005149
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.023732
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.037462
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.001978
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.005980
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.003890


Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.077807
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.144579
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.006916

Test set: Average loss: 0.0289, Accuracy: 9905/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.003240
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.024976
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.022388
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.001376
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.059894
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.014698
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.005471
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.003527
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.012304
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.002834
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.037269
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.026388
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.049698
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.001846
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.006028
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.003008
Train Epoch: 8 [10

Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.000979
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.008997
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.004417
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.002257
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.112463
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.088693
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.009209
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.009664
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.001387
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.030965
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.026981
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.073232
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.002981
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.002596
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.063343
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.060741
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.010038
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.063417
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.113611
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.000871


Train Epoch: 11 [24960/60000 (42%)]	Loss: 0.026620
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.012087
Train Epoch: 11 [26240/60000 (44%)]	Loss: 0.046077
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.032465
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.001095
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.004311
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.060771
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.056878
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.026464
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.013406
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.023990
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.150542
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.002247
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.076157
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.002413
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.081016
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.090274
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.000840
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.044674
Train Epoch: 11 [37120/60000 (6

Train Epoch: 13 [7040/60000 (12%)]	Loss: 0.003447
Train Epoch: 13 [7680/60000 (13%)]	Loss: 0.002991
Train Epoch: 13 [8320/60000 (14%)]	Loss: 0.007310
Train Epoch: 13 [8960/60000 (15%)]	Loss: 0.002020
Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.009553
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.046222
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.142071
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.016046
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.015151
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.029305
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.065655
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.009832
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.005977
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.003856
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.001719
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.010395
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.046760
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.010247
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.002268
Train Epoch: 13 [19200/60000 (32%)]	

Train Epoch: 14 [50560/60000 (84%)]	Loss: 0.013724
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.039021
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.032960
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.005646
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.018760
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.167726
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.069760
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.018405
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.001575
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.005181
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.003400
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.001082
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.014494
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.000419
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.000764

Test set: Average loss: 0.0265, Accuracy: 9918/10000 (99%)



In [34]:
import torch.onnx

In [31]:
x = torch.randn(16, 1, 28, 28, requires_grad=True).to(device)

In [32]:
torch.onnx.export(model,
                  x,
                  "mnist.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names=['input'],
                  output_names=['output'],
                  dynamic_axes={
                      'input': {0: 'batch_size'},
                      'output': {0: 'batch_size'},
                  })

In [33]:
import onnx

In [ ]:
model = onnx.load("mnist.onnx")

In [ ]:
onnx.checker.check_model(model)